In [ ]:
samples = []
# change path to fastq files
filepath = ''

# file containing R1 (read1) fastq filenames, one file per line
with open('microc_fastqs.txt', 'r') as f:
    for line in f:
        r1_filename = line.strip('\n')
        r2_filename = r1_filename.replace('R1_001', 'R2_001')
        metadata = r1_filename.split('_')
        sample = metadata[3]
        rep = metadata[4]
        lane = metadata[7]
        id = '_'.join([sample, rep, lane])
        samples.append([id, sample, rep, lane, filepath + r1_filename, filepath + r2_filename])

In [18]:
with open('microc_sample_map.csv', 'w') as f:
    f.write('sample_id,sample,rep,lane,fastq_r1,fastq_r2\n')
    for i in samples:
        f.write(','.join(i) + '\n')

### Parse statistics file from pairtools

In [ ]:
def parse_stats(filename):
    """
    Parse stats files from pairtools stats
    """
    out = {}
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip('\n').split('\t')
            if line[0] == 'total_mapped':
                out['mapped'] = int(line[1])
            elif line[0] == 'total_dups':
                out['dups'] = int(line[1])
            elif line[0] == 'total_nodups':
                out['nodups'] = int(line[1])
            elif line[0] == 'cis':
                out['cis'] = int(line[1])
                break
            else:
                pass

    return out

In [2]:
from collections import defaultdict

In [ ]:
# write alignment stats to file
parsed_stats = open('all_pairtools_dup_stats.txt', 'w')
parsed_stats.write('sample\tmapped\tnodups\tdups\tperc_nodup\n')
counts = defaultdict(list)

with open('microc_stats_files.txt', 'r') as f:
    for line in f:
        sample = '\t'.join(line.split('_')[:2])
        celltype = line.split('_')[0]
        stats = parse_stats('/home/gridsan/chong/hansen_lab_shared/clarice/microc_061224/merged_per_rep/' + line.strip('\n'))
        counts[celltype].append(stats['cis'])
        percentage_nodup = stats['nodups']/stats['mapped']
        parsed_stats.write('{sample}\t{mapped:,}\t{nodups:,}\t{dups:,}\t{perc:.2f}\n'.format(
            sample = sample, mapped = stats['mapped'], nodups = stats['nodups'], dups = stats['dups'],
            perc = percentage_nodup
        ))

parsed_stats.close()

In [ ]:
[sum(v) for k, v in counts.items()]

[5147917512, 4225600203, 5511141755]